<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #06



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>

<a id='prob'></a>
## Problem #06

<br>
Facilities <br>
$|I| = 100$ 
<br>
<br>
Clients <br>
$|J| = 100$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [1]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time


# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Facilities Fixed Costs
def getFacilities_STRCapacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(file_list[i+shift])
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost


# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/MO1'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))




In [2]:
print(num_facilities, num_customers)
print(shipping_cost)

100 100
{(0, 0): 2.37627, (0, 1): 5.31695, (0, 2): 5.57118, (0, 3): 5.83591, (0, 4): 9.15027, (0, 5): 5.82114, (0, 6): 5.7796, (0, 7): 5.67768, (0, 8): 4.49169, (0, 9): 6.72159, (0, 10): 3.6284, (0, 11): 9.12322, (0, 12): 6.32598, (0, 13): 3.48818, (0, 14): 3.58675, (0, 15): 3.73334, (0, 16): 3.52031, (0, 17): 8.07946, (0, 18): 8.12691, (0, 19): 7.39602, (0, 20): 3.0342, (0, 21): 9.79799, (0, 22): 2.8878, (0, 23): 5.30806, (0, 24): 9.6352, (0, 25): 6.56233, (0, 26): 8.73168, (0, 27): 2.95125, (0, 28): 9.33563, (0, 29): 8.25158, (0, 30): 4.8502, (0, 31): 5.71954, (0, 32): 5.70687, (0, 33): 3.77907, (0, 34): 9.07588, (0, 35): 9.85327, (0, 36): 8.22727, (0, 37): 5.55128, (0, 38): 3.55556, (0, 39): 3.33675, (0, 40): 9.82005, (0, 41): 6.9682, (0, 42): 3.9418, (0, 43): 9.41557, (0, 44): 8.73737, (0, 45): 2.62152, (0, 46): 4.25865, (0, 47): 7.3897, (0, 48): 3.94682, (0, 49): 6.84679, (0, 50): 3.59799, (0, 51): 7.33873, (0, 52): 7.17666, (0, 53): 8.28582, (0, 54): 2.80267, (0, 55): 5.43553, (0

In [12]:

start = time.time()

# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()

end = time.time()
print("TIME IS: ",end - start)

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 10100 rows, 10100 columns and 30000 nonzeros
Model fingerprint: 0xeafaad1f
Variable types: 10000 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.445903e+07
Presolve time: 0.15s
Presolved: 10100 rows, 10100 columns, 30000 nonzeros
Variable types: 10000 continuous, 100 integer (100 binary)

Root relaxation: objective 1.656083e+03, 5005 iterations, 0.17 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1656.0826600 1656.08266  0.00%    

In [4]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

#m.write('UFLP_06_Simplex.lp')


Build a warehouse at location 24.

Build a warehouse at location 53.

Client 1 receives 100.0 % of its demand  from Warehouse 24 .

Client 2 receives 100.0 % of its demand  from Warehouse 24 .

Client 3 receives 100.0 % of its demand  from Warehouse 53 .

Client 4 receives 100.0 % of its demand  from Warehouse 24 .

Client 5 receives 100.0 % of its demand  from Warehouse 53 .

Client 6 receives 100.0 % of its demand  from Warehouse 53 .

Client 7 receives 100.0 % of its demand  from Warehouse 53 .

Client 8 receives 100.0 % of its demand  from Warehouse 24 .

Client 9 receives 100.0 % of its demand  from Warehouse 53 .

Client 10 receives 100.0 % of its demand  from Warehouse 53 .

Client 11 receives 100.0 % of its demand  from Warehouse 24 .

Client 12 receives 100.0 % of its demand  from Warehouse 53 .

Client 13 receives 100.0 % of its demand  from Warehouse 53 .

Client 14 receives 100.0 % of its demand  from Warehouse 53 .

Client 15 receives 100.0 % of its demand  from Warehouse

<br>
<br>

<a id='random'></a>
### Random Heuristic 


In [5]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random


start = time.time()

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



end = time.time()
print("TIME IS: ",end - start)

#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[0, 2, 4, 5, 6, 8, 9, 13, 15, 18, 20, 21, 22, 24, 26, 29, 30, 31, 32, 34, 35, 36, 37, 38, 42, 43, 47, 48, 50, 51, 53, 54, 55, 57, 60, 61, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 77, 78, 80, 83, 84, 85, 86, 87, 89, 90, 91, 93, 94, 95, 97]
10929138.876250008
TIME IS:  0.0021889209747314453


In [6]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 1.

Build a warehouse at location 3.

Build a warehouse at location 5.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 9.

Build a warehouse at location 10.

Build a warehouse at location 14.

Build a warehouse at location 16.

Build a warehouse at location 19.

Build a warehouse at location 21.

Build a warehouse at location 22.

Build a warehouse at location 23.

Build a warehouse at location 25.

Build a warehouse at location 27.

Build a warehouse at location 30.

Build a warehouse at location 31.

Build a warehouse at location 32.

Build a warehouse at location 33.

Build a warehouse at location 35.

Build a warehouse at location 36.

Build a warehouse at location 37.

Build a warehouse at location 38.

Build a warehouse at location 39.

Build a warehouse at location 43.

Build a warehouse at location 44.

Build a warehouse at location 48.

Build a warehouse at location 49.

Build a warehouse at loca

In [7]:
len(facs)

63

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [8]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

[1, 2, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 19, 21, 24, 27, 28, 31, 33, 34, 35, 36, 39, 40, 42, 43, 44, 45, 47, 49, 50, 51, 52, 53, 54, 57, 59, 61, 62, 63, 64, 68, 69, 70, 71, 72, 74, 75, 76, 79, 80, 82, 83, 84, 85, 88, 91, 92, 94, 95, 97, 99]
13187216.330819994
65
TIME IS:  0.008944034576416016


<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [9]:
#Get the greatest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

[0, 2, 3, 4, 6, 7, 8, 11, 14, 15, 17, 20, 22, 26, 27, 29, 30, 34, 35, 36, 38, 40, 41, 42, 43, 44, 45, 48, 52, 53, 55, 58, 59, 60, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 83, 84, 88, 89, 90, 91, 92, 93, 94, 96, 98, 99]
12758289
12761052.005059993
62
TIME IS:  0.010008096694946289


<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [10]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

[23]
50
6877.00736
1
TIME IS:  0.01856207847595215


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [11]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

end = time.time()
print("TIME IS: ",end - start)

[33]
994707
100467164.77985005
1
TIME IS:  0.014225006103515625
